In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma-lcs/generated_results_Gemma_Lcs.csv
/kaggle/input/nir-generated-answers/generated_results_with_f1_llama2_chegeka.csv
/kaggle/input/nir-generated-answers/chgk_generated_results_with_f1_mistral.csv
/kaggle/input/nir-generated-answers/chgk_generated_results_Gemma.csv
/kaggle/input/nir-generated-answers/chgk_generated_results_tinyLlama.csv
/kaggle/input/nir-generated-answers/chgk_generated_results_with_f1_gemma.csv
/kaggle/input/nir-generated-answers/generated_results_Gemma_Lcs.csv
/kaggle/input/nir-generated-answers/chegeka-2ndsemester-nir (2).ipynb
/kaggle/input/nir-generated-answers/lcs_vikhr_generated_results.csv
/kaggle/input/nir-generated-answers/generated_results_llama2_chegeka.csv
/kaggle/input/nir-generated-answers/chgk_generated_results_mistral.csv
/kaggle/input/nir-generated-answers/lcs_llama2_generated_results.csv
/kaggle/input/nir-generated-answers/generated_results_Vikhr_7B_chegeka.csv
/kaggle/input/nir-generated-answers/lcs_tinyllama_generated_results.csv

# Models - Chegeka

1. Gemma
2. Mistral
3. Llama 2
4. Vikhr
5. Tiny LLama

In [2]:
!pip install langchain
!pip install peft
!pip install trl

!pip install -q -U transformers
!pip install -q accelerate
!pip install -q -i https://pypi.org/simple/ bitsandbytes
!pip install -q -U datasets
!pip install shap

import warnings
warnings.filterwarnings("ignore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the so

In [3]:
gm = "/kaggle/input/gemma/transformers/2b/2"
mistral = '/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1'
llama2 = '/kaggle/input/llama-2/pytorch/7b-hf/1'
vikhr = 'Vikhrmodels/Vikhr-7B-instruct_merged'
tinyllama = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'

In [4]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

import torch
import torch.nn as nn

import transformers
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from datasets import Dataset
from peft import LoraConfig, PeftConfig
import bitsandbytes as bnb
from trl import SFTTrainer

from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

2024-05-10 13:02:30.776525: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-10 13:02:30.776670: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-10 13:02:30.954135: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Gemma

In [5]:
from transformers import pipeline
pipe = pipeline("text-generation", model=gm)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Mistral

In [6]:
from transformers import pipeline

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model_mistral = AutoModelForCausalLM.from_pretrained(
    mistral,
    device_map="auto",
    quantization_config=bnb_config, 
)

model_mistral.config.use_cache = False
model_mistral.config.pretraining_tp = 1

max_seq_length = 2048

tokenizer = AutoTokenizer.from_pretrained(mistral)
# EOS_TOKEN = tokenizer.eos_token

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
pipe = pipeline("text-generation", 
                model=model_mistral, 
                tokenizer = tokenizer)

# Llama 2

In [ ]:
from transformers import pipeline

compute_dtype = getattr(torch, "float16")

llama2 = '/kaggle/input/llama-2/pytorch/7b-hf/1'
# llama2 = 'meta-llama/Llama-2-7b'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model_llama2 = AutoModelForCausalLM.from_pretrained(
    llama2,
    device_map="auto",
    quantization_config=bnb_config, 
)

model_llama2.config.use_cache = False
model_llama2.config.pretraining_tp = 1

max_seq_length = 2048

tokenizer = AutoTokenizer.from_pretrained(llama2)

In [ ]:
pipe = pipeline("text-generation", 
                model=model_llama2, 
                tokenizer = tokenizer)

# Vikhr

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.prompts import PromptTemplate
from transformers import pipeline

vikhr = 'Vikhrmodels/Vikhr-7B-instruct_0.4'
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model_vikhr = AutoModelForCausalLM.from_pretrained(
    vikhr,
    device_map="auto",
    quantization_config=bnb_config, 
)

model_vikhr.config.use_cache = False
model_vikhr.config.pretraining_tp = 1

max_seq_length = 2048

tokenizer = AutoTokenizer.from_pretrained(vikhr)
# EOS_TOKEN = tokenizer.eos_token

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", 
                model=model_vikhr, 
                tokenizer= tokenizer, 
                )

# TinyLlama

In [ ]:
from transformers import pipeline
pipe = pipeline("text-generation", model=tinyllama)

# Data sampling

In [ ]:
from datasets import load_dataset

dataset = load_dataset("ai-forever/MERA", 'lcs')

In [ ]:
import random

total_prompts = dataset["public_test"].num_rows
random_indices = random.sample(range(total_prompts), 10)

random_prompts = []
inputs = []
outputs = []

for idx in random_indices:
  prompt_text = dataset["public_test"][idx]
  inputs_text = dataset['public_test'][idx]['inputs']
  outputs_text = dataset['public_test'][idx]['outputs']  

  random_prompts.append(prompt_text)
  inputs.append(inputs_text)
  outputs.append(outputs_text)


# Print the 10 random prompts
print("10 Random Prompts from MERA 'lcs' dataset:")
for prompt in random_prompts:
  print(prompt)

In [ ]:
lcs_task_collection = [
    {'input': "CZUWAQSJOS LTHWJNYRZ"},
    {'input': "LONUPTDGMYUOTLSKOEPSVRUT SVTZVMLSGZWOWSMPHZYYFVRERFYXUI"},
    {'input': "HZCEEHBLMDTZFLBQZJV NPRNUVHPXBGGACZJHZXQPFLU"},
    {'input': "VRJOZGMJQTSFQCRZAEUKJRWJICVLQMD MXVOYOFUURVUYXJMCLLVFXXT"},
    {'input': "TZIWMBYEVSDHKOVAIXTZ DUAPNZHVFSVIPGLEMNFOJ"},
    {'input': "FFOBNUN ZPSXDKMEYDVCRXMMYPWCMKVTQKH"},
    {'input': "LBKLB DNXMDOHDA"},
    {'input': "PZWLQBSQWQNELMGOVWRNOXS WIWUSEVJKKPQXRWVEDJVTMHQODQJ"},
    {'input': "NIKVSXCMRUQWJNMFVLOEKDOYFZZUSDJ JPJUPOALIYCCHYA"},
    {'input': "ZJYYBHRZD QZLVSBBRFYUWUZADYYAPTEEIGKG"}
]


lcs_collection_answers = [
    {'answer': "2"},
    {'answer': "9"},
    {'answer': "6"},
    {'answer': "8"},
    {'answer': "5"},
    {'answer': "0"},
    {'answer': "0"},
    {'answer': "7"},
    {'answer': "3"},
    {'answer': "5"}
]



In [ ]:
from langchain.prompts import PromptTemplate

lcs_prompt_collection = [
    PromptTemplate(
        template='''Даны две строки в качестве входных данных:\n"{inputs}"\nУкажите длину их наибольшей общей подпоследовательности. Между буквами могут быть пропуски. Запишите только цифру, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно. Ответ: ''',
        input_variables= ['input']
    ),
    PromptTemplate(
        template='''По заданным строкам:\n"{inputs}"\nОпределите длину наибольшей общей подпоследовательности. Учтите возможность пропуска символов. Запишите только цифру, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно. Ответ: ''',
        input_variables=['input']
    ),
    PromptTemplate(
        template='''Даны две строки: "{inputs}". Укажите длину наибольшей общей подпоследовательности. Пропуск символов допустим. Запишите только цифру, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно. Ответ: ''',
        input_variables=['input']
    ),
    PromptTemplate(
        template='''По двум введенным строкам:\n"{inputs}"\nУкажите длину наибольшей общей подпоследовательности. Разрешены пропуски символов. Запишите только цифру, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно. Ответ: ''',
        input_variables=['input']
    ),
    PromptTemplate(
        template='''Предоставлены строки:\n"{inputs}"\nНайдите длину наибольшей общей подпоследовательности. Можно использовать пропуск символов. Запишите только цифру, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно. Ответ: ''',
        input_variables=['input']
    ),
    PromptTemplate(
        template='''Заданы две строки:\n"{inputs}"\nОпределите, сколько символов составляет наибольшая общая подпоследовательность. Пропуск символов допустим. Запишите только цифру, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно. Ответ: ''',
        input_variables=['input']
    ),
    PromptTemplate(
        template='''По данным строкам:\n"{inputs}"\nНайдите длину наибольшей общей подпоследовательности. Разрешено использование пропусков. Запишите только цифру, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно. Ответ: ''',
        input_variables=['input']
    ),
    PromptTemplate(
        template='''У вас есть две строки:\n"{inputs}"\nНеобходимо определить длину наибольшей общей подпоследовательности. Пропуски символов разрешены. Запишите только цифру, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно. Ответ: ''',
        input_variables=['input']
    ),
    PromptTemplate(
        template='''По строкам:\n"{inputs}"\nУкажите, сколько символов составляет наибольшая общая подпоследовательность. Можно использовать пропуск символов. Запишите только цифру, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно. Ответ: ''',
        input_variables=['input']
    ),
    PromptTemplate(
        template='''Две строки предоставлены:\n"{inputs}"\nНайдите длину наибольшей общей подпоследовательности. Разрешается пропуск символов. Запишите только цифру, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно. Ответ: ''',
        input_variables=['input']
    ),
]


In [ ]:
import pandas as pd

def process_texts(prompt_collection, task_collection, answer_collection, pipe):
    results = []

    for task, answer in zip(task_collection, answer_collection):
        for prompt_template in prompt_collection:
            prompt_text = prompt_template.template.format(inputs=task['input'])
            generated_text = pipe(prompt_text, return_full_text=False, max_new_tokens=100)[0]['generated_text']
            print("Generated Text:", generated_text)  
            results.append({'question': task['input'], 'prompt': prompt_text, 'expected_answer': answer['answer'], 'generated_text': generated_text})

    df_results = pd.DataFrame(results)
    df_results.to_csv('generated_results.csv', index=False)

In [ ]:
# Gemma generated Answers

process_texts(lcs_prompt_collection, lcs_task_collection, lcs_collection_answers, pipe)

In [ ]:
# Mistral generated Answers

process_texts(lcs_prompt_collection, lcs_task_collection, lcs_collection_answers, pipe)

In [ ]:
# Llama2 generated Answers

process_texts(lcs_prompt_collection, lcs_task_collection, lcs_collection_answers, pipe)

In [ ]:
# Vikhr generated Answers

process_texts(lcs_prompt_collection, lcs_task_collection, lcs_collection_answers, pipe)

In [ ]:
# TinyLLama generated Answers

process_texts(lcs_prompt_collection, lcs_task_collection, lcs_collection_answers, pipe)

In [ ]:
import pandas as pd

df_results = pd.read_csv('/kaggle/working/generated_results.csv')
df_results.head()

In [ ]:
df_results.shape

In [ ]:
import nltk

nltk.download('punkt')

# Calculating Accuracy